In [ ]:
using HDF5
using PyPlot
using PyCall
@pyimport scipy.interpolate as si

In [ ]:
fid=h5open("diagonalization.h5","r")
#fid2=h5open("twoflux.h5","r")

In [ ]:
names(fid)

In [ ]:
h=fid["50/5"]
names(h)

In [ ]:
Xb=read(h["Xb"]);
Xw=read(h["Xw"]);
E=read(h["E"]);
S=read(h["S"]);
U=read(h["U"]);
V=read(h["V"]);
Bondsb=read(h["Bondsb"]);
i0b=read(h["i0b"]);

In [ ]:
Nx=50;
N=3*Nx^2;

In [ ]:
site=round(Int,N/2);
n=150;
omega=collect(linspace(0,6,150));
heights2=zeros(omega);

  for  j in 1:N;
        k=round(Int,ceil(E[j]*50));
        heights2[k]+=.5*V[site,j].^2
  end

In [ ]:
data=readdlm("digi.dat")

In [ ]:
pygui(false)
plot(omega,heights,label="Me")
plot(omega[1:50],data[2:end]*.013/510,color="green",label="Scaled Paper")
xlim(0,2)
title("Comparison of Paper and me")
xlabel("Omega")
legend(loc="Upper Left")

In [ ]:
pygui(true)
scatter(E[1:100],U[round(Int,N/2),1:100].^2,color="red")
scatter(E[1:100]+.0001,U[round(Int,N/2)+1,1:100].^2)
scatter(E[1:100]+.0002,U[round(Int,N/2)+2,1:100].^2,color="green")
ylim(-.001,.002)

In [ ]:
x=cat(1,Xb,Xw)[:,1];
y=cat(1,Xb,Xw)[:,2];
pygui(false)
scatter(x,y)
minimum(x)

In [ ]:
xmin=86;
xmat=repmat(collect(-xmin:.25:xmin),1,8*xmin+1);
ymat=repmat(transpose(collect(-xmin:.25:xmin)),8*xmin+1,1);

In [ ]:
grid0=si.griddata(cat(1,Xb,Xw),cat(1,U[:,end],V[:,end]),(xmat,ymat),method="nearest");
grid1=si.griddata(cat(1,Xb,Xw),cat(1,U[:,end-1],V[:,end-1]),(xmat,ymat),method="nearest");
grid2=si.griddata(cat(1,Xb,Xw),cat(1,U[:,end-2],V[:,end-2]),(xmat,ymat),method="nearest");
grid3=si.griddata(cat(1,Xb,Xw),cat(1,U[:,end-3],V[:,end-3]),(xmat,ymat),method="nearest");
grids=si.griddata(cat(1,Xb,Xw),cat(1,U[:,1],V[:,1]),(xmat,ymat),method="nearest");

In [ ]:
grid0=si.griddata(Xb,U[:,1],(xmat,ymat),method="nearest");

In [ ]:
pcolor(grid0)

In [ ]:
pygui(true)
pcolor(xmat,ymat,grid0.^2,cmap="coolwarm")
colorbar()
#scatter(X[:,1],X[:,2],marker=".",alpha=0.05)

In [ ]:
pygui(false)
pcolor(xmat,ymat,grid1.^2,cmap="coolwarm")
colorbar()
scatter(X[:,1],X[:,2],marker=".",alpha=0.05)

In [ ]:
pygui(false)
pcolor(xmat,ymat,grid2.^2,cmap="coolwarm")
colorbar()
scatter(x,y,marker=".",alpha=0.05)

In [ ]:
pygui(false)
pcolor(xmat,ymat,grid3.^2,cmap="coolwarm")
colorbar()
scatter(x,y,marker=".",alpha=0.05)

In [ ]:
pygui(false)
pcolor(xmat,ymat,grids.^2,cmap="coolwarm")
colorbar()
scatter(X[:,1],X[:,2],marker=".",alpha=0.05)

In [ ]:
X=cat(1,Xb,Xw);
U2=cat(2,U,V);

trans1=copy(X)
trans2=copy(X)
trans3=copy(X)

trans1[:,2]+=-Nx*3;

trans2[:,1]+=sqrt(3)*Nx*3/2;
trans2[:,2]+=-Nx*3/2;

trans3[:,1]+=-sqrt(3)*Nx*3/2;
trans3[:,2]+=-Nx*3/2;

In [ ]:
function mindist(i::Int,j::Int)
    dist(x,y)=sqrt(x^2+y^2);
    
    r0=dist(X[i,1]-X[j,1],X[i,2]-X[j,2]);
    r1=dist(trans1[i,1]-X[j,1],trans1[i,2]-X[j,2]);
    r2=dist(trans2[i,1]-X[j,1],trans2[i,2]-X[j,2]);
    r3=dist(trans3[i,1]-X[j,1],trans3[i,2]-X[j,2]);
    
    r4=dist(trans1[j,1]-X[i,1],trans1[j,2]-X[i,2]);
    r5=dist(trans2[j,1]-X[i,1],trans2[j,2]-X[i,2]);
    r6=dist(trans3[j,1]-X[i,1],trans3[j,2]-X[i,2]);
    
    return minimum([r0,r1,r2,r3,r4,r5,r6])
end

In [ ]:
j=606;
db=zeros(Float64,N);
for i in 1:N
    db[i]=mindistb(i,j);
end

In [ ]:
scatter(Xb[:,1],Xb[:,2],c=db,marker="o",s=10,linewidths=0)
colorbar()

In [ ]:
rs=zeros(Float64,2*N)
j=1000;
for i in 1:2*N
    rs[i]=mindist(i,j);
end

In [ ]:
p=sortperm(rs)
plot(rs[p[1:50]],marker="o")

In [ ]:
plot(squeeze(U2[end-1,p[1:50]]*U2[end-1,1],1))

In [ ]:
bins=zeros(Float64,2*87);
nbins=zeros(Float64,2*87)
for j in 1:2*N;
    for i in 1:(j-1)
        r=ceil(Int,2*mindist(i,j));
        if r==0
            r=1
        end
        bins[r]+=(U2[end,i]*U2[end,j])^2;
        nbins[r]+=1;
    end
    if j%1000==0
        println("at ",j," ",j/N)
    end
end
binsp=bins./(nbins);
for i in 1:2*87
    if isnan(binsp[i])
        binsp[i]=binsp[i+1];
    end
end

In [ ]:
bins1=zeros(Float64,2*87);
nbins=zeros(Float64,2*87)
for j in 1:2*N;
    for i in 1:(j-1)
        r=ceil(Int,2*mindist(i,j));
        if r==0
            r=1
        end
        bins1[r]+=(U2[end-1,i]*U2[end-1,j])^2;
        nbins[r]+=1;
    end
    if j%1000==0
        println("at ",j," ",j/N)
    end
end
binsp1=bins1./(nbins);
for i in 1:2*87
    if isnan(binsp1[i])
        binsp1[i]=binsp1[i+1];
    end
end

In [ ]:
scatter(x-1,binsp1[1:50]+binsp[1:50]-2*mean(U2[end,:].^2).^2,color="black",marker="o",linewidth=0)
ylim(-1e-9,1e-9)

In [ ]:
x=collect(1.:50.);

In [ ]:
pygui(false)
ylim(0,1e-9)
scatter(x-1,abs(binsp[1:50]-mean(U2[end,:].^2)^2),marker="o",color="red")#c=nbins[1:50],linewidth=0)
scatter(x-1,abs(binsp1[1:50]-mean(U2[end-1,:].^2)^2),marker="o",linewidth=0)
scatter(x-1,abs(binsp1[1:50]+binsp[1:50]-2*mean(U2[end,:].^2+U2[end-1,:].^2)^2),color="black",marker="o",linewidth=0)
#plot(binsp[1:40]-mean(U2[end-1,:].^2)^2)
plot(x,zeros(x))

In [ ]:
plot(nbins[1:40])

In [ ]:
for i in 1:2*87
    if binsp[i]==0#isnan(binsp[i])
        binsp[i]=binsp[i+1];
    end
end

In [ ]:
mean(U2[end-1,:])

In [ ]:
mean(binsp)

In [ ]:
plot(E[end:-1:end-4],marker="o")

In [ ]:
mean(U2[end,:].^2),mean(U2[end-1,:].^2),mean(U2[end-2,:].^2)